◆役割
あなたはpythonのプログラマのスペシャリスト
LLMの実装に習熟している

◆仕様
顧客情報を要約させるチャットを作りたい
RAGは使わない

プロンプト案を複数作成する
DSPYを使ってプロンプトを最適化する
顧客情報を取り込む（サンプルデータでよい）
LLMに連携し、要約結果を得る
要約結果が簡潔であるか、わかりやすいか、一貫性があるかをLLMasaJudgeでテストする
LLMの回答は再現性がないため、tempertureを0にしたうえで、3回施行し、その結果の平均値を評価する
評価の関数はDSpyのevaluateでカスタムの評価関数をさk巣栄する
施行の結果をMLFlowで

OpenAIのAPIを利用
モデル：gpt4-o-mini
APIキー：XXXXX（別途設定）

まずはノートブックで実行したい
動作が確認できてからクラス化、関数化をするので、それを意識した処理に分けてほしい


In [0]:
# 必要なライブラリのインストール
!pip install dspy-ai openai mlflow pandas numpy

In [0]:
dbutils.library.restartPython() 

In [0]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot
import openai
import mlflow
import pandas as pd
import numpy as np
from typing import List, Dict
import os
from datetime import datetime

# MLflow設定
mlflow.openai.autolog()
mlflow.set_experiment(experiment_id=2762244084694123)

In [0]:
# 顧客情報サンプルデータ
sample_customers = [
    {
        "id": "C001",
        "name": "山田太郎",
        "age": 35,
        "occupation": "会社員",
        "purchase_history": "過去6ヶ月で3回購入。主に電化製品を購入。平均購入額は5万円。",
        "inquiries": "配送に関する問い合わせが2件。製品の使い方に関する質問が1件。",
        "notes": "リピート顧客。メールマガジン購読中。次回購入時10%割引クーポン保有。"
    },
    {
        "id": "C002",
        "name": "佐藤花子",
        "age": 28,
        "occupation": "自営業",
        "purchase_history": "初回購入。書籍とオフィス用品を購入。購入額は1.2万円。",
        "inquiries": "返品ポリシーに関する問い合わせが1件。",
        "notes": "新規顧客。SNS経由での流入。レビュー投稿の依頼中。"
    },
    {
        "id": "C003",
        "name": "鈴木一郎",
        "age": 42,
        "occupation": "経営者",
        "purchase_history": "VIP顧客。月に2-3回購入。高額商品を好む傾向。累計購入額200万円超。",
        "inquiries": "VIPサポート窓口利用。製品カスタマイズの相談が複数回。",
        "notes": "優良顧客。専任担当者あり。法人契約も検討中。誕生日月に特別オファー送付済み。"
    },
    {
        "id": "C004",
        "name": "田中美咲",
        "age": 31,
        "occupation": "看護師",
        "purchase_history": "過去1年で5回購入。健康関連商品とコスメを主に購入。平均購入額3万円。",
        "inquiries": "製品の成分に関する詳細な問い合わせが3件。アレルギー対応の確認。",
        "notes": "健康志向の高い顧客。定期購入プラン加入中。友人紹介プログラム利用歴あり。"
    },
    {
        "id": "C005",
        "name": "高橋健二",
        "age": 55,
        "occupation": "公務員",
        "purchase_history": "2年前から顧客。年に4-5回購入。趣味関連の商品を購入。平均購入額4万円。",
        "inquiries": "配送遅延に関するクレームが1件。その後のフォローアップで満足度回復。",
        "notes": "安定した購入パターン。ポイントプログラム活用。過去にクレームあったが現在は良好な関係。"
    }
]

# データフレーム化
df_customers = pd.DataFrame(sample_customers)
print("サンプルデータ件数:", len(df_customers))
print("\n最初の顧客データ:")
print(df_customers.iloc[0].to_dict())

In [0]:
# OpenAI LM設定(temperature=0で再現性を確保)
lm = dspy.LM(
    model='gpt-4o-mini',
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.0,
    max_tokens=500
)

dspy.settings.configure(lm=lm)

# 顧客情報要約のシグネチャ定義
class CustomerSummarySignature(dspy.Signature):
    """顧客情報を簡潔かつ明確に要約します。"""
    
    customer_id = dspy.InputField(desc="顧客ID")
    name = dspy.InputField(desc="顧客名")
    age = dspy.InputField(desc="年齢")
    occupation = dspy.InputField(desc="職業")
    purchase_history = dspy.InputField(desc="購入履歴")
    inquiries = dspy.InputField(desc="問い合わせ履歴")
    notes = dspy.InputField(desc="備考")
    
    summary = dspy.OutputField(desc="顧客情報の要約(200文字以内、簡潔で分かりやすく一貫性のある内容)")

In [0]:
# プロンプト案1: 基本的な要約
class BasicSummaryModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.summarize = dspy.ChainOfThought(CustomerSummarySignature)
    
    def forward(self, customer_id, name, age, occupation, purchase_history, inquiries, notes):
        result = self.summarize(
            customer_id=customer_id,
            name=name,
            age=age,
            occupation=occupation,
            purchase_history=purchase_history,
            inquiries=inquiries,
            notes=notes
        )
        return result

# プロンプト案2: 構造化された要約
class StructuredSummaryModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.summarize = dspy.Predict(CustomerSummarySignature)
    
    def forward(self, customer_id, name, age, occupation, purchase_history, inquiries, notes):
        # 構造化されたプロンプト指示を追加
        structured_notes = f"{notes}\n\n要約形式: [顧客属性] [購入傾向] [注意事項]の順で記載"
        result = self.summarize(
            customer_id=customer_id,
            name=name,
            age=age,
            occupation=occupation,
            purchase_history=purchase_history,
            inquiries=inquiries,
            notes=structured_notes
        )
        return result

# プロンプト案3: ビジネス重視の要約
class BusinessFocusedSummaryModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.summarize = dspy.ChainOfThought(CustomerSummarySignature)
    
    def forward(self, customer_id, name, age, occupation, purchase_history, inquiries, notes):
        # ビジネス視点を強調
        business_notes = f"{notes}\n\n重点: 顧客価値、リスク、次回アクション提案を含める"
        result = self.summarize(
            customer_id=customer_id,
            name=name,
            age=age,
            occupation=occupation,
            purchase_history=purchase_history,
            inquiries=inquiries,
            notes=business_notes
        )
        return result

print("3つのプロンプトモジュールを定義完了")

In [0]:
# 評価用のLLM設定(Judge用は少し高めのtemperatureも可)
judge_lm = dspy.LM(
    model='gpt-4o-mini',
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.0,
    max_tokens=200
)

# 評価シグネチャ
class EvaluationSignature(dspy.Signature):
    """要約の品質を評価します。"""
    
    summary = dspy.InputField(desc="評価対象の要約文")
    criteria = dspy.InputField(desc="評価基準")
    
    score = dspy.OutputField(desc="スコア(0-10の整数)")
    reasoning = dspy.OutputField(desc="評価理由")

# 評価モジュール
class SummaryJudge(dspy.Module):
    def __init__(self):
        super().__init__()
        with dspy.context(lm=judge_lm):
            self.evaluate = dspy.Predict(EvaluationSignature)
    
    def forward(self, summary, criteria):
        with dspy.context(lm=judge_lm):
            result = self.evaluate(summary=summary, criteria=criteria)
        return result

judge = SummaryJudge()

# カスタム評価関数(簡潔性、明瞭性、一貫性を評価)
def evaluate_summary_quality(example, prediction, trace=None):
    """
    要約の品質を3つの観点で評価
    - 簡潔性: 適切な長さで無駄がないか
    - 明瞭性: 理解しやすく明確か
    - 一貫性: 情報が矛盾なく整理されているか
    """
    summary = prediction.summary if hasattr(prediction, 'summary') else str(prediction)
    
    # 3つの観点で評価
    criteria_list = [
        "簡潔性: 要約は適切な長さで、冗長な表現がなく、要点が絞られているか(10点満点)",
        "明瞭性: 要約は分かりやすく、専門用語の説明があり、読みやすいか(10点満点)",
        "一貫性: 情報が矛盾なく、論理的に整理され、一貫した視点で記述されているか(10点満点)"
    ]
    
    scores = []
    reasonings = []
    
    for criteria in criteria_list:
        result = judge(summary=summary, criteria=criteria)
        try:
            score = int(result.score)
            scores.append(score)
            reasonings.append(result.reasoning)
        except:
            scores.append(0)
            reasonings.append("評価エラー")
    
    # 平均スコアを計算(0-10のスケール)
    avg_score = np.mean(scores)
    
    # DSPy評価用に0-1のスケールに正規化
    normalized_score = avg_score / 10.0
    
    return normalized_score, {
        "conciseness_score": scores[0],
        "clarity_score": scores[1],
        "consistency_score": scores[2],
        "average_score": avg_score,
        "reasonings": reasonings
    }

print("評価関数を定義完了")

In [0]:
def evaluate_with_multiple_trials(module, customer_data, num_trials=3):
    """
    同じ入力で3回試行し、結果の平均を取る
    """
    all_summaries = []
    all_scores = []
    all_details = []
    
    for trial in range(num_trials):
        # 要約生成
        prediction = module(
            customer_id=customer_data['id'],
            name=customer_data['name'],
            age=str(customer_data['age']),
            occupation=customer_data['occupation'],
            purchase_history=customer_data['purchase_history'],
            inquiries=customer_data['inquiries'],
            notes=customer_data['notes']
        )
        
        # 評価
        score, details = evaluate_summary_quality(customer_data, prediction)
        
        all_summaries.append(prediction.summary)
        all_scores.append(score)
        all_details.append(details)
        
        print(f"  試行 {trial + 1}: スコア = {details['average_score']:.2f}/10")
    
    # 平均計算
    avg_normalized_score = np.mean(all_scores)
    avg_raw_score = np.mean([d['average_score'] for d in all_details])
    
    return {
        "summaries": all_summaries,
        "normalized_scores": all_scores,
        "avg_normalized_score": avg_normalized_score,
        "avg_raw_score": avg_raw_score,
        "trial_details": all_details
    }

print("複数試行評価関数を定義完了")

In [0]:
# 評価結果を保存する辞書
evaluation_results = {}

# 3つのモジュールを初期化
modules = {
    "basic": BasicSummaryModule(),
    "structured": StructuredSummaryModule(),
    "business_focused": BusinessFocusedSummaryModule()
}

# 各モジュールを評価
for module_name, module in modules.items():
    print(f"\n{'='*60}")
    print(f"モジュール評価開始: {module_name}")
    print(f"{'='*60}")
    
    module_results = []
    
    # 各顧客データで評価
    for idx, customer in enumerate(sample_customers):
        print(f"\n顧客 {idx + 1}/{len(sample_customers)}: {customer['name']}")
        
        result = evaluate_with_multiple_trials(module, customer, num_trials=3)
        result['customer_id'] = customer['id']
        result['customer_name'] = customer['name']
        module_results.append(result)
    
    evaluation_results[module_name] = module_results
    
    # モジュール全体の平均スコア
    overall_avg = np.mean([r['avg_raw_score'] for r in module_results])
    print(f"\n{module_name} 全体平均スコア: {overall_avg:.2f}/10")

print("\n\n全モジュールの評価完了")

In [0]:
# 各モジュールの結果をMLflowに記録
for module_name, results in evaluation_results.items():
    with mlflow.start_run(run_name=f"prompt_{module_name}"):
        # パラメータ記録
        mlflow.log_param("module_type", module_name)
        mlflow.log_param("model", "gpt-4o-mini")
        mlflow.log_param("temperature", 0.0)
        mlflow.log_param("num_trials", 3)
        mlflow.log_param("num_customers", len(results))
        
        # 全体メトリクス
        overall_avg_score = np.mean([r['avg_raw_score'] for r in results])
        overall_conciseness = np.mean([
            np.mean([d['conciseness_score'] for d in r['trial_details']]) 
            for r in results
        ])
        overall_clarity = np.mean([
            np.mean([d['clarity_score'] for d in r['trial_details']]) 
            for r in results
        ])
        overall_consistency = np.mean([
            np.mean([d['consistency_score'] for d in r['trial_details']]) 
            for r in results
        ])
        
        mlflow.log_metric("overall_avg_score", overall_avg_score)
        mlflow.log_metric("overall_conciseness", overall_conciseness)
        mlflow.log_metric("overall_clarity", overall_clarity)
        mlflow.log_metric("overall_consistency", overall_consistency)
        
        # 顧客別メトリクス
        for idx, result in enumerate(results):
            mlflow.log_metric(f"customer_{idx+1}_avg_score", result['avg_raw_score'])
        
        # 結果の詳細をアーティファクトとして保存
        results_df = pd.DataFrame([
            {
                'customer_id': r['customer_id'],
                'customer_name': r['customer_name'],
                'avg_score': r['avg_raw_score'],
                'summary_1': r['summaries'][0],
                'summary_2': r['summaries'][1],
                'summary_3': r['summaries'][2]
            }
            for r in results
        ])
        
        results_df.to_csv(f"{module_name}_results.csv", index=False)
        mlflow.log_artifact(f"{module_name}_results.csv")
        
        print(f"\n{module_name} の結果をMLflowに記録完了")

print("\n全ての結果をMLflowに記録完了")

In [0]:
# 結果の比較表を作成
comparison_data = []

for module_name, results in evaluation_results.items():
    overall_avg = np.mean([r['avg_raw_score'] for r in results])
    conciseness = np.mean([
        np.mean([d['conciseness_score'] for d in r['trial_details']]) 
        for r in results
    ])
    clarity = np.mean([
        np.mean([d['clarity_score'] for d in r['trial_details']]) 
        for r in results
    ])
    consistency = np.mean([
        np.mean([d['consistency_score'] for d in r['trial_details']]) 
        for r in results
    ])
    
    comparison_data.append({
        'モジュール': module_name,
        '総合スコア': f"{overall_avg:.2f}",
        '簡潔性': f"{conciseness:.2f}",
        '明瞭性': f"{clarity:.2f}",
        '一貫性': f"{consistency:.2f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("プロンプト案の比較結果")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# 最良のモジュールを特定
best_module = max(
    evaluation_results.items(),
    key=lambda x: np.mean([r['avg_raw_score'] for r in x[1]])
)
print(f"\n最良のプロンプト案: {best_module[0]}")
print(f"スコア: {np.mean([r['avg_raw_score'] for r in best_module[1]]):.2f}/10")

In [0]:
# 最良のモジュールでのサンプル要約を表示
print("\n" + "="*80)
print(f"【{best_module[0]}】の要約サンプル")
print("="*80)

for idx, result in enumerate(best_module[1][:2]):  # 最初の2顧客のみ表示
    print(f"\n顧客: {result['customer_name']} ({result['customer_id']})")
    print(f"平均スコア: {result['avg_raw_score']:.2f}/10")
    print(f"\n要約例(試行1):")
    print(result['summaries'][0])
    print(f"\n評価詳細:")
    print(f"  簡潔性: {result['trial_details'][0]['conciseness_score']}/10")
    print(f"  明瞭性: {result['trial_details'][0]['clarity_score']}/10")
    print(f"  一貫性: {result['trial_details'][0]['consistency_score']}/10")
    print("-" * 80)